# MASLite tutorial: A Trio of Agent, Scheduler and Messages
This is an interactive tutorial on MASLite, the corner stone of Dematic sim engine, which is aimed to help bring new developers up to speed quickly.
A side note to help you understand MASLite better. There are different types of simulation modeling technique. If you come from the Simpy world, then you know that Simpy is an Process Based Modeling.
MASLite, however, is Agent Based Modeling. Although, there is no clear boundary between one another, the two modeling techniques pivot on different mindsets.


<img src="figs/trio_of_agent_scheduler_and_message.png" width="50%" height="50%">

In [1]:
from maslite import Agent, Scheduler, AgentMessage, AlarmRegistry
from collections import deque
import random

Below is how you can create an agent,
one thing to keep in mind is the UUID, each agent needs to have a "universal unique identifier", that is basically your name, SSN, or home address, those things "legally" define who you are.
by default, all agents will be assigned with a UUID at initialization based on the sequence of their creation, which is an integer

In [2]:
agent_a = Agent()
agent_b = Agent()
assert isinstance(agent_a.uuid, int) and isinstance(agent_b.uuid, int), 'default uuid in maslite agent should be integer'
assert agent_b.uuid > agent_a.uuid, 'agent_b is created after agent_a, thus it uuid should be larger'
print(agent_a.uuid, agent_b.uuid)

1 2


You can give an uuid to initiate an agent, as long as it is hashable

In [3]:
agent_a = Agent('James Bound')
assert agent_a.uuid == 'James Bound', 'you can change the uuid at initialization'
print("Agent's UUID: ", agent_a.uuid)

Agent's UUID:  James Bound


Agent A can send Message to Agent B, message should contain sender's uuid (Agent A's uuid) and receiver's uuid (Agent B's uuid)
However, to send the Message, Agents need the help of Scheduler
To take advantage of Scheduler, you need to initialize it, and also register the agents to the same scheduler

In [4]:
agent_a = Agent('007') # keep in mind we decided to call James by code number
agent_b = Agent('009')
scheduler = Scheduler()  # initialize the Scheduler
scheduler.add(agent_a)  # add (register) the agent to scheduler
scheduler.add(agent_b)  # add (register) the agent to scheduler

Registering agent Agent 007
007 subscribing to 007 on all topics.
007 subscribing to Agent for all agents.
Registering agent Agent 009
009 subscribing to 009 on all topics.
009 subscribing to Agent for all agents.


Below is how you send a message,
Just like how you send a letter in the old times
You will need a delivery address (receiver.uuid) and a return address (sender.uuid)
Good news is in the Kingdom of MASlitevile, the mail delivery service is free, no stamp needed!

<img src="figs/send_a_letter.jpg" width="30%" height="50%">

In [5]:
msg = AgentMessage(sender=agent_a.uuid, receiver=agent_b.uuid)

After Agent A sends the message, Agent B does not receive that immediately.
The message should go to Scheduler's mail_queue, which is a deque, just like the mailbox in front of the post office, in the old times

<img src="figs/old_mail_box.jpg" width="20%" height="20%">

In [6]:
agent_a.send(msg)
assert isinstance(scheduler.mail_queue[0], AgentMessage)
assert scheduler.mail_queue[0].sender == '007' and scheduler.mail_queue[0].receiver == '009'

Scheduler will dispatch the message to agents via function process_mail_queue,
the message will be delivered to inbox of Agent B.
Note the Scheduler uses Agent uuid to find the right recipient.

<img src="figs/post_dog.jpg" width="20%" height="20%">

In [7]:
assert len(agent_b.inbox) == 0
scheduler.process_mail_queue()
assert isinstance(agent_b.inbox[0], AgentMessage)
assert agent_b.inbox[0].sender=='007' and agent_b.inbox[0].receiver =='009'

now that the message has been delivered,
Agent B needs to get the message from its inbox

<img src="figs/mail_box.jpg" width="30%" height="30%">

In [8]:
msg = agent_b.receive()
assert msg.sender == '007' and msg.receiver == '009'
assert len(agent_b.inbox) == 0

Now the message has been successfully delivered from Agent A (007) to Agent B (009), but then what?
Let us say, we want Agent B to "laugh" after received the message from Agent A.
To achieve this, we need to do three things:
1. give AgentMessage a topic, so that Agent B can recognize this specific message, see "topic" in agent message
2. register the function 'laugh' in Agent B, see "self.operations" below
3. After Scheduler delivered the messages to Agent B's inbox, Agent B needs to read those message one by one and react to it, see "self.update"

In [9]:
class TestAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations['A funny joke'] = self.laugh

    def update(self):
        while self.messages:
            msg = self.receive()  # you receive the msg in update
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def laugh(self, msg):
        print(f"{msg.receiver} is laughing")


Let's create two new agents and add them to a scheduler

In [10]:
agent_a = TestAgent(uuid='007')
agent_b = TestAgent(uuid='009')
scheduler = Scheduler()
scheduler.add(agent_a)
scheduler.add(agent_b)

Registering agent TestAgent 007
007 subscribing to 007 on all topics.
007 subscribing to TestAgent for all agents.
Registering agent TestAgent 009
009 subscribing to 009 on all topics.
009 subscribing to TestAgent for all agents.


Now let Agent A send a message that is "A funny joke" to Agent B

In [11]:
msg = AgentMessage(sender=agent_a, receiver=agent_b, topic='A funny joke')
agent_a.send(msg)
scheduler.process_mail_queue()
agent_b.update()

009 is laughing


See, Agent B is laughing!

<img src="figs/laugh.png" width="10%" height="10%">

What if Agent A sends something else, like a picture of a dog? or even just a bare message no topic, an empty envelope?

In [12]:
msg = AgentMessage(sender=agent_a, receiver=agent_b, topic='A picture of a dog')
agent_a.send(msg)
scheduler.process_mail_queue()
agent_b.update()

don't know what to do with A picture of a dog


In [13]:
msg = AgentMessage(sender=agent_a, receiver=agent_b)
agent_a.send(msg)
scheduler.process_mail_queue()
agent_b.update()

don't know what to do with AgentMessage


See, Agent B does not know what to do with "A picture of a dog", why?
Because you did not register "A picture of a dog" with any function in "self.operations".
Now, very important thing to remember if you want a message to trigger a function of an agent:
The message topic must be registered as the key, and the function must be registered as value in **opertaions**, which is a dictionary

<img src="figs\message_function_pair.png" width="50%" height="50%">

if you look closely, when you don't give any topic, the printout simply says "don't know what to do with AgentMessage"
Is that a coincidence that the topic is the class name?
No! In maslite, if you don't give topic as input, then the topic will be set as class name of the message by default, see "__init__" of AgentMessage in maslite
This brings us a more powerful way of sending messages, see below

In [14]:
class Joke(AgentMessage):

    def __init__(self, sender, receiver):
        super().__init__(sender, receiver)


class TestAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations[Joke.__name__] = self.laugh

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def laugh(self, msg):
        print(f"{msg.receiver} is laughing")


In [15]:
agent_a = TestAgent(uuid='007')
agent_b = TestAgent(uuid='009')
scheduler = Scheduler()
scheduler.add(agent_a)
scheduler.add(agent_b)

Registering agent TestAgent 007
007 subscribing to 007 on all topics.
007 subscribing to TestAgent for all agents.
Registering agent TestAgent 009
009 subscribing to 009 on all topics.
009 subscribing to TestAgent for all agents.


In [16]:
msg = Joke(sender=agent_a, receiver=agent_b)
assert msg.topic == Joke.__name__
agent_a.send(msg)
scheduler.process_mail_queue()
agent_b.update()

009 is laughing


**Take Home Question: why this method is more powerful?**

Agent B finds the joke funny and laughed, and he wants to be nice, so thinks he should write a joke to Agent A as well.
Agent A feels the same, then...

In [17]:
class Joke(AgentMessage):

    def __init__(self, sender, receiver):
        super().__init__(sender, receiver)


class TestAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations[Joke.__name__] = self.laugh

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def laugh(self, msg):
        print(f"{msg.receiver} is laughing")
        self.send(Joke(sender=self.uuid, receiver=msg.sender))

In [18]:
agent_a = TestAgent(uuid='007')
agent_b = TestAgent(uuid='009')
scheduler = Scheduler()
scheduler.add(agent_a)
scheduler.add(agent_b)

Registering agent TestAgent 007
007 subscribing to 007 on all topics.
007 subscribing to TestAgent for all agents.
Registering agent TestAgent 009
009 subscribing to 009 on all topics.
009 subscribing to TestAgent for all agents.


In [19]:
msg = Joke(sender=agent_a, receiver=agent_b)
agent_a.send(msg)
scheduler.process_mail_queue()
agent_b.update()

009 is laughing


In [20]:
scheduler.process_mail_queue()
agent_a.update()

007 is laughing


In [21]:
scheduler.process_mail_queue()
agent_b.update()

009 is laughing


Now, 007 and 009 are going to keep laughing as long as scheduler_4 is alive...
This is the magical chain reaction!

<img src="figs/action_reaction.gif" width="50%" height="50%">

In certain cases, the whole system can be triggered by only one message

<img src="figs/dominos.gif" width="30%" height="30%">

Another trick is that an agent can send a message to its own as well!
007 never stops laughing

<img src="figs/infinity.gif" width="20%" height="20%">

In [22]:
msg = Joke(sender=agent_a, receiver=agent_a)
agent_a.send(msg)
scheduler.process_mail_queue()
agent_a.update()
scheduler.process_mail_queue()
agent_a.update()

007 is laughing
007 is laughing
007 is laughing


Now, the two agents are getting alone with each other, one day agents A decides to send a gift card that worth $ 20 to Agent B together with the regular jokes...

In [23]:
class Joke(AgentMessage):

    def __init__(self, sender, receiver, gift_card_dollars):
        super().__init__(sender, receiver)
        self.gift_card_dollars = gift_card_dollars


class TestAgent(Agent):

    def __init__(self, uuid, dollars):
        super().__init__(uuid=uuid)
        self.dollars = dollars
        self.operations[Joke.__name__] = self.receive_joke

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def send_joke(self, gift_card_dollars, agent_uuid):
        self.dollars -= gift_card_dollars
        self.send(Joke(sender=self.uuid, receiver=agent_uuid, gift_card_dollars=gift_card_dollars))

    def receive_joke(self, msg):
        print(f"{msg.receiver} is laughing")
        self.dollars += msg.gift_card_dollars
        gift_card_dollars = random.randint(0, self.dollars)
        self.send_joke(gift_card_dollars, msg.sender)

In [24]:
agent_a = TestAgent(uuid='007', dollars=10000)
agent_b = TestAgent(uuid='009', dollars=10000)
scheduler = Scheduler()
scheduler.add(agent_a)
scheduler.add(agent_b)

Registering agent TestAgent 007
007 subscribing to 007 on all topics.
007 subscribing to TestAgent for all agents.
Registering agent TestAgent 009
009 subscribing to 009 on all topics.
009 subscribing to TestAgent for all agents.


In [25]:
agent_a.send_joke(2000, agent_b.uuid)
scheduler.process_mail_queue()
agent_b.update()

009 is laughing


Not sure if Agent B is laughing because of the joke or money

<img src="figs/counting_money.gif" width="40%" height="40%">

Now lets check the dollars

In [26]:
assert agent_a.dollars == 8000
money_agent_b_sent = scheduler.mail_queue[0].gift_card_dollars
assert agent_b.dollars == 10000 + 2000 - money_agent_b_sent
scheduler.process_mail_queue()
agent_a.update()
assert agent_a.dollars == 10000 - 2000 + money_agent_b_sent - scheduler.mail_queue[0].gift_card_dollars

007 is laughing


Hope you learned two tricks
1. use attributes of AgentMessage to carry important information (money, of course)
2. pairwise receive and send functions (only if you are willing to give money back, of course)

Now, Agent A finds herself very good at telling jokes after practicing with Agent B, she wants to start a magazine business, the magazine's name is "50 jokes you must know"
Agent B's friends Agent C, D, E, F are all very interested in reading and laughing, Agent A can continue to send jokes one by one.
But Scheduler offers another powerful service, **Broadcast**

To create a broadcast, we need to create a broadcast message. A broadcast message is different from a direct communication message, **it has a topic and no receiver**.
For broadcast message to work, we must implement a suitable **copy** method.
Make sure you are copying the right thing, otherwise you will find yourself in a wierd bug

In [27]:
class FunnyJokesMagazine(AgentMessage):
    def __init__(self, sender, topic):
        super().__init__(sender=sender, topic=topic)

    def copy(self):
        return FunnyJokesMagazine(sender=self.sender, topic=self.topic)


Broadcasts are subscription based. Therefore, for Agent B and his friends to enjoy Agent A's magazine, each agent must subscribe first. see the **setup** function

In [28]:
class TestAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations["50 jokes you must know"] = self.laugh

    def setup(self):
        self.subscribe(topic="50 jokes you must know")

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def laugh(self, msg):
        print(f"Agent {self.uuid} is laughing")

In [29]:
agent_a = TestAgent('A')
agent_b = TestAgent('B')
agent_c = TestAgent('C')
agent_d = TestAgent('D')
agent_e = TestAgent('E')
agent_f = TestAgent('F')

In [30]:
scheduler = Scheduler()
for agent in [agent_a, agent_b, agent_c, agent_d, agent_e, agent_f]:
    scheduler.add(agent)

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
A subscribing to 50 jokes you must know for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
B subscribing to 50 jokes you must know for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
C subscribing to 50 jokes you must know for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
D subscribing to 50 jokes you must know for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.
E subscribing to 50 jokes you must know for all agents.
Registering agent TestAgent F
F subscribing to F on all topics.
F subscribing to TestAgent for all agents.
F subscribing to 50 jokes you must know for all agents.


In [31]:
msg = FunnyJokesMagazine(sender=agent_a.uuid, topic="50 jokes you must know")
agent_a.send(msg)
scheduler.process_mail_queue()

Now Agent B and all his friends are enjoying Agent A's funny jokes using Broadcast.

In [32]:
for agent in [agent_a, agent_b, agent_c, agent_d, agent_e, agent_f]:
    assert isinstance(agent.inbox[0], FunnyJokesMagazine)
    agent.update()

Agent A is laughing
Agent B is laughing
Agent C is laughing
Agent D is laughing
Agent E is laughing
Agent F is laughing


Now let's understand more about the concept of default topic in AgentMessage. To understand this, consider the below scenario

Agent A's jokes are widely popular in the Kingdom of MASlite. Majority of agents subscribed to Agent A's Joke service.
Recently a couple of new agents have migrated from other kingdom into MASlite, and they came to know about Agent A's jokes popularity.
There are different types of joke services (AgentMessage topic) which Agent A provides such as DailyJokes, WeeklyJokes, MonthlyTopTenJokes etc.
New residents are confused witch joke service (AgentMessage topic) to subscribe, so Agent A recommends them to subscribe default joke service (AgentMessage with no specific topic)
which entertains its subscribers with a new joke every Monday and this default service is free of cost.
So the new residents happily agreed and took the default joke subscription (AgentMessage with no specific topic).

In [33]:
class DefaultJokesService(AgentMessage):

    def __init__(self, sender, topic=None):
        super().__init__(sender, sender, topic)

    def copy(self):
        return DefaultJokesService(sender=self.sender, topic=self.topic)


class SubscriberAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations[DefaultJokesService.__name__] = self.thank_you

    def setup(self):
        self.subscribe(DefaultJokesService.__name__)

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def thank_you(self, msg):
        print(f"Thank you Agent {msg.sender}, this is funny! I want to subscribe to more jokes")
        self.subscribe("50 funny jokes you should know")
        self.operations["50 funny jokes you should know"] = self.laugh
        
    def laugh(self, msg):
        print(f"Agent {self.uuid} is laughing")


In [34]:
agent_a = Agent('A')
agent_x = SubscriberAgent('New X')
agent_y = SubscriberAgent('New Y')
agent_z = SubscriberAgent('New Z')

In [35]:
scheduler = Scheduler()
for agent in [agent_a, agent_x, agent_y, agent_z]:
    scheduler.add(agent)

Registering agent Agent A
A subscribing to A on all topics.
A subscribing to Agent for all agents.
Registering agent SubscriberAgent New X
New X subscribing to New X on all topics.
New X subscribing to SubscriberAgent for all agents.
New X subscribing to DefaultJokesService on all topics.
Registering agent SubscriberAgent New Y
New Y subscribing to New Y on all topics.
New Y subscribing to SubscriberAgent for all agents.
New Y subscribing to DefaultJokesService on all topics.
Registering agent SubscriberAgent New Z
New Z subscribing to New Z on all topics.
New Z subscribing to SubscriberAgent for all agents.
New Z subscribing to DefaultJokesService on all topics.


If Agent A sends "50 funny jokes you should know" as the topic of DefaultJokesService, then none of the newly migrated people would receive anything.

In [36]:
msg = DefaultJokesService(sender=agent_a.uuid, topic = '50 funny jokes you should know')
print(msg.topic)
agent_a.send(msg)
scheduler.process_mail_queue()
for agent in [agent_x, agent_y, agent_z]:
    assert len(agent.inbox) == 0

50 funny jokes you should know


If Agent A does not include "50 funny jokes you should know" as the topic of DefaultJokesService, then all of the newly migrated people would receive that, and laugh, and decide to subscribe to Agent A's 50 funny jokes you should know

In [37]:
msg = DefaultJokesService(sender=agent_a.uuid)
print(msg.topic)
agent_a.send(msg)
scheduler.process_mail_queue()
for agent in [agent_x, agent_y, agent_z]:
    agent.update()

New X subscribing to 50 funny jokes you should know on all topics.
New Y subscribing to 50 funny jokes you should know on all topics.
New Z subscribing to 50 funny jokes you should know on all topics.


DefaultJokesService
Thank you Agent A, this is funny! I want to subscribe to more jokes
Thank you Agent A, this is funny! I want to subscribe to more jokes
Thank you Agent A, this is funny! I want to subscribe to more jokes


Now if agent sends '50 funny jokes you should know', they will laugh

In [38]:
msg = DefaultJokesService(sender=agent_a.uuid, topic='50 funny jokes you should know')
print(msg.topic)
agent_a.send(msg)
scheduler.process_mail_queue()
for agent in [agent_x, agent_y, agent_z]:
    agent.update()

50 funny jokes you should know
Agent New X is laughing
Agent New Y is laughing
Agent New Z is laughing


**remember the importance of topic in the agent message**

Time flies, and it is the year **1984** for Maslite Kingdom...
The residents are reading jokes, however, big brother is watching...

<img src="figs/1984.jpg" width="25%" height="25%">

In [39]:
class Joke(AgentMessage):

    def __init__(self, sender, receiver):
        super().__init__(sender, receiver)

    def copy(self):
        return Joke(self.sender, self.receiver)


class TestAgent(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations[Joke.__name__] = self.laugh

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def laugh(self, msg):
        print(f"{msg.receiver} is laughing")


class BigBrother(Agent):

    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        self.operations[Joke.__name__] = self.read

    def setup(self):
        self.subscribe(Joke.__name__)

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def read(self, msg):
        print(f"{self.uuid} says to {msg.sender} and {msg.receiver}: Big Brother is watching you!")


In [40]:
agent_a = TestAgent(uuid='007')
agent_b = TestAgent(uuid='009')
agent_c = BigBrother(uuid='KGB')
scheduler = Scheduler()
scheduler.add(agent_a)
scheduler.add(agent_b)
scheduler.add(agent_c)

Registering agent TestAgent 007
007 subscribing to 007 on all topics.
007 subscribing to TestAgent for all agents.
Registering agent TestAgent 009
009 subscribing to 009 on all topics.
009 subscribing to TestAgent for all agents.
Registering agent BigBrother KGB
KGB subscribing to KGB on all topics.
KGB subscribing to BigBrother for all agents.
KGB subscribing to Joke on all topics.


In [41]:
msg = Joke(sender=agent_a.uuid, receiver=agent_b.uuid)
agent_a.send(msg)
scheduler.process_mail_queue()
for agent in scheduler.agents.values():
    agent.update()

009 is laughing
KGB says to 007 and 009: Big Brother is watching you!


Maslite has a hidden feature, an agent can subscribe to the "private" message between other agents without the consent of the designated sender and receiver

<img src="figs/fear.png" width="5%" height="5%">

Now we covered the basics of what you should master in maslite, let us go pro!
It seems what we have seen so far happens all at once, and to run a simulation, we missed a very important concept **Time**

<img src="figs/quote_about_time.png" width="40%" height="40%">

**How does time flow in the kingdom of MASlite?** AKA **How does MASlite advance the clock for simulation?**
set alarm!

<img src="figs/alarm.gif" width="20%" height="20%">

To understand the concept of set_alarm in MASlite, let's consider the following scenario.
Agent A is giving amazon coupon (10 - 100) dollars to all his **PopularJokesMagazine** subscribers during the summer sale in MASlite Kingdom.
The coupon is sent to the subscribers 5 seconds after they receive the magazine.
Don't ask how Agent A how does she do the timing so accurate, it is in MASlite Kingdom, ask the king.

In [42]:
class PopularJokesMagazine(AgentMessage):

    def __init__(self, sender, receiver, topic=None):
        super().__init__(sender, receiver, topic)


class TwentyDollarCoupon(AgentMessage):

    def __init__(self, sender, receiver, topic=None):
        super().__init__(sender, receiver, topic)
        self.coupon = random.randint(10, 100)


class TestAgent(Agent):

    def __init__(self, uuid, coupon_cnt=0, coupon_val=0):
        super().__init__(uuid=uuid)
        self.coupon_cnt = coupon_cnt
        self.coupon_val = coupon_val
        self.operations[PopularJokesMagazine.__name__] = self.thank_you
        self.operations[TwentyDollarCoupon.__name__] = self.coupon_received

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            if operation is not None:
               operation(msg)
            else:
                print(f"don't know what to do with {msg.topic}")

    def thank_you(self, msg):
        print(f"Thank you Agent {msg.sender}, looking forward to your jokes and our coupon.")

    def coupon_received(self, msg):
        self.coupon_val = msg.coupon
        self.coupon_cnt += 1
        print(f"Thank you, I have received my ${self.coupon_val} coupon, now I have {self.coupon_cnt} coupons")

**Note that scheduler uses real time by default**

In [43]:
agent_a = TestAgent('A')
agent_b = TestAgent('B')
agent_c = TestAgent('C')
agent_d = TestAgent('D')
scheduler = Scheduler(real_time=False)
for agent in [agent_a, agent_b, agent_c, agent_d]:
    scheduler.add(agent)

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.


We can interpret the alarm as a message to the future receiver, or a pre-defined fate to an agent in the Kingdom of Maslite...

<img src="figs/message_to_future_self.jpg" width="25%" height="25%">

in another word, the agent will receive such message in the future, after "alarm_time" (check the function **set_alarm**).
But where is the alarm message when you set it?
As we did not see it in the mail queue for sure. And the mail queue only contains the message that needs to be sent at current time

In [44]:
msg = PopularJokesMagazine(sender=agent_a.uuid, receiver=agent_b.uuid)
agent_a.send(msg)
agent_a.set_alarm(alarm_time=5, alarm_message=TwentyDollarCoupon(sender=agent_a.uuid, receiver=agent_b.uuid)) # ignore_alarm_if_idle=False
assert len(scheduler.mail_queue) == 1
assert isinstance(scheduler.mail_queue[0], PopularJokesMagazine)

In the clock of Scheduler, there is an attribute called "registry", which is a dictionary.
It contains the receiver's uuid as the key, and an object called "AlarmRegistry".
The AlarmRegistry has an attribute "alarm" which is also a dictionary.
It contains the alarm time and a list of messages that such receiver should receive

In [45]:
assert isinstance(scheduler.clock.registry, dict)
assert agent_b.uuid in scheduler.clock.registry
assert isinstance(scheduler.clock.registry[agent_b.uuid], AlarmRegistry)
alarm_registry = scheduler.clock.registry[agent_b.uuid]
assert isinstance(alarm_registry.alarms, dict)
assert 5 in alarm_registry.alarms
list_of_messages_to_send_at_5_sec = alarm_registry.alarms[5]
assert isinstance(list_of_messages_to_send_at_5_sec[0], TwentyDollarCoupon)
assert len(list_of_messages_to_send_at_5_sec) == 1

Now, shall we move the clock forward?
Tick Tock

<img src="figs/clock_tick_tock.gif" width="25%" height="25%">

In [46]:
print(scheduler.clock.time)
scheduler.clock.tick()
print(scheduler.clock.time)

0
0


Wait a second, why the clock cannot move forward?
Because you are not done at time 0, you still have message to deliver!

In [47]:
scheduler.process_mail_queue()
agent_b.update()
assert len(scheduler.mail_queue) == 0

Thank you Agent A, looking forward to your jokes and our coupon.


In [48]:
print(scheduler.clock.time)
scheduler.clock.tick()
print(scheduler.clock.time)

0
0


Wait a second, why the clock still cannot move forward?
Because you still have agents that needs update!
Wait a second, we just updated agent "B"!
Because when scheduler "add" an agent at the beginning, it added the agent to “needs_update" by default, and you should clear that attribute after you have done with the update
**Take Home Question, why does scheduler needs to do that?**

In [49]:
assert scheduler.needs_update == {'A', 'B', 'C', 'D'}
scheduler.needs_update.clear()
print(scheduler.clock.time)
scheduler.clock.tick()
print(scheduler.clock.time)

0
5


The clock finally moves!
Now the time is at 5, would agent_b get his coupon as what agent a planned?
Not yet, the message is still stored in the alarm registry, remember? We need to release them.

In [50]:
assert len(scheduler.mail_queue) == 0
scheduler.clock.release_alarm_messages()
assert len(scheduler.mail_queue) == 1
assert isinstance(scheduler.mail_queue[0], TwentyDollarCoupon)

Now the message is in the mail queue, the scheduler just need to dispatch the message and the agent needs to update it

In [51]:
scheduler.process_mail_queue()
agent_b.update()

Thank you, I have received my $95 coupon, now I have 1 coupons


Let us have a deep dive in how MASLite Scheduler handles multiple alarms, and how the event are "scheduled".
Let us think a scenario more similar to nuclear fission.
We create a TestAgent (think this as a neutron), and a TestMessage (think this as an atom). 
Whenever the TestAgent (atom) receives a TestMessage (neutron), such agent will randomly select agents (including itself, not very scientific, I know) and set an alarm with TestMessage random alarm time (minimum 0, maximum 5) for random times (minimum 0, maximum 3)

In [52]:
class TestMessage(AgentMessage):
    
    def __init__(self, sender, receiver, created_time, scheduled_alarm_time):
        super().__init__(sender, receiver)
        self.created_time = created_time
        self.scheduled_alarm_time = scheduled_alarm_time
    
    def __repr__(self):
        return f'From {self.sender} to {self.receiver}, created at {self.created_time}, scheduled alarm time {self.scheduled_alarm_time}'
    
    def __str__(self):
         return f'From {self.sender} to {self.receiver}, created at {self.created_time}, scheduled alarm time {self.scheduled_alarm_time}'
        
class TestAgent(Agent):
    
    agents = []
    
    def __init__(self, uuid):
        super().__init__(uuid=uuid)
        assert uuid not in TestAgent.agents, "cannot have duplicated uuid"
        TestAgent.agents.append(uuid)
        self.operations[TestMessage.__name__] = self.receive_test_message

    def update(self):
        while self.messages:
            msg = self.receive()
            operation = self.operations.get(msg.topic)
            operation(msg)

    def receive_test_message(self, msg):
        number_of_alarms = random.randint(0, 3) # randomly decide how many alarm message to send, maximum 3, minimum 0
        print(f"Agent {self.uuid} receives alarm {str(msg)}, decides to set {number_of_alarms} alarms")
        for _ in range(number_of_alarms):
            agent_to_receive_alarm = TestAgent.agents[random.randint(0, len(TestAgent.agents) - 1)] # randomly select an agent to receive the alarm
            alarm_time = random.randint(0, 5) # randomly decide what the alarm time should be
            self.set_alarm(alarm_time, TestMessage(self.uuid, agent_to_receive_alarm, 
                                                   self._scheduler_api.clock.time, alarm_time))
            print(f'An alarm has been set by Agent {self.uuid} at t = {self._scheduler_api.clock.time}, '
                  f'alarm time is {alarm_time}, receiver is Agent {agent_to_receive_alarm}')

In [53]:
random.seed(1) # set up a random seed
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0, 0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [54]:
# iteration 1
scheduler.process_mail_queue()
agents[0].update()

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 1 alarms
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent E


In [55]:
assert scheduler.clock.registry.keys() == {'E'}, 'the agents that have alarms (fate) scheduled (destinied) will be shown here'
assert 0 in scheduler.clock.registry['E'].alarms, "t = 0 should be added to Agent E's alarms"
assert scheduler.clock.alarm_time == [0], "alarm time t = 0 has been added to the 'schedule' (fate)"

Be careful with the data structure in **scheduler.clock.registry[agent.uuid].alarms**

In [56]:
print(scheduler.clock.registry['E'].alarms[0])
print(scheduler.clock.registry['E'].alarms[0][0])

[From A to E, created at 0, scheduled alarm time 0]
From A to E, created at 0, scheduled alarm time 0


**Iteration 1**: the primed message sent from A to A, triggers the function "receive_test_message". Agent A decides to set 1 alarm for agent E with 0 as alarm time

<img src="figs/Iteration1.PNG" width="70%" height="70%">

**Guess what would happen in iteration 2**?

In [57]:
# iteration 2
assert scheduler.clock.time == 0
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 0
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'E'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  E
Agent E receives alarm From A to E, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent A
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent D


In [58]:
assert scheduler.clock.registry.keys() == {'E', 'A', 'D'}
assert scheduler.clock.registry['E'].alarms.keys() == set(), 'Although E is in registry, but it alarms should be an empty dict'
print(scheduler.clock.registry['A'].alarms[3])
print(scheduler.clock.registry['A'].alarms[3][0])
print(scheduler.clock.registry['D'].alarms[3])
print(scheduler.clock.registry['D'].alarms[3][0])

[From E to A, created at 0, scheduled alarm time 3]
From E to A, created at 0, scheduled alarm time 3
[From E to D, created at 0, scheduled alarm time 3]
From E to D, created at 0, scheduled alarm time 3


**Iteration 2**, since the alarm time set by A in previous iteration is 0, the clock does not move forward. Agent E decides to set 1 alarm for both agent A & D in 3 seconds

<img src="figs/Iteration2.PNG" width="70%" height="70%">

In [59]:
# iteration 3
assert scheduler.clock.time == 0
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 3
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'A', 'D'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  D
Agent D receives alarm From E to D, created at 0, scheduled alarm time 3, decides to set 3 alarms
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent B
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent D
An alarm has been set by Agent D at t = 3, alarm time is 3, receiver is Agent D
Agent  A
Agent A receives alarm From E to A, created at 0, scheduled alarm time 3, decides to set 0 alarms


**Iteration 3**

<img src="figs/Iteration3.PNG" width="70%" height="70%">

In [60]:
# iteration 4
assert scheduler.clock.time == 3
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 3
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'B', 'D'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  B
Agent B receives alarm From D to B, created at 3, scheduled alarm time 0, decides to set 3 alarms
An alarm has been set by Agent B at t = 3, alarm time is 5, receiver is Agent C
An alarm has been set by Agent B at t = 3, alarm time is 4, receiver is Agent B
An alarm has been set by Agent B at t = 3, alarm time is 2, receiver is Agent A
Agent  D
Agent D receives alarm From D to D, created at 3, scheduled alarm time 0, decides to set 0 alarms


**Iteration 4** similar to what happened at iteration 2, the clock did not move forward

<img src="figs/Iteration4.PNG" width="70%" height="70%">

In [61]:
# iteration 5
assert scheduler.clock.time == 3
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 5
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'A'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  A
Agent A receives alarm From B to A, created at 3, scheduled alarm time 2, decides to set 0 alarms


**Iteration 5** Although the alarm at t = 5 was added later than that at t = 6, but since the event is scheduled based on time, the clock advanced to t = 5

<img src="figs/Iteration5.PNG" width="70%" height="70%">

In [62]:
 # iteration 6
assert scheduler.clock.time == 5
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 6
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'D'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  D
Agent D receives alarm From D to D, created at 3, scheduled alarm time 3, decides to set 0 alarms


**Iteration 6** 

<img src="figs/Iteration6.PNG" width="70%" height="70%">

In [63]:
# iteration 7
assert scheduler.clock.time == 6
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 7
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'B'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  B
Agent B receives alarm From B to B, created at 3, scheduled alarm time 4, decides to set 0 alarms


**Iteration 7**

<img src="figs/Iteration7.PNG" width="70%" height="70%">

In [64]:
# iteration 8
assert scheduler.clock.time == 7
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 8
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'C'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  C
Agent C receives alarm From B to C, created at 3, scheduled alarm time 5, decides to set 3 alarms
An alarm has been set by Agent C at t = 8, alarm time is 3, receiver is Agent B
An alarm has been set by Agent C at t = 8, alarm time is 4, receiver is Agent A
An alarm has been set by Agent C at t = 8, alarm time is 3, receiver is Agent B


**Iteration 8**

<img src="figs/Iteration8.PNG" width="70%" height="70%">

In [65]:
# iteration 9
assert scheduler.clock.time == 8
scheduler.needs_update.clear()  # first clear the needs_update
scheduler.clock.tick() # advance the clock the next event time
assert scheduler.clock.time == 11
scheduler.clock.release_alarm_messages()  # release the alarm messages
scheduler.process_mail_queue() # deliver the messages to designated receiver/subscriber
assert scheduler.needs_update == {'B'}
for agent_id in scheduler.needs_update:  # calls the agent to update
    print("Agent ", agent_id)
    agent = scheduler.agents[agent_id]
    agent.update()

Agent  B
Agent B receives alarm From C to B, created at 8, scheduled alarm time 3, decides to set 3 alarms
An alarm has been set by Agent B at t = 11, alarm time is 1, receiver is Agent E
An alarm has been set by Agent B at t = 11, alarm time is 1, receiver is Agent C
An alarm has been set by Agent B at t = 11, alarm time is 3, receiver is Agent B
Agent B receives alarm From C to B, created at 8, scheduled alarm time 3, decides to set 2 alarms
An alarm has been set by Agent B at t = 11, alarm time is 3, receiver is Agent A
An alarm has been set by Agent B at t = 11, alarm time is 5, receiver is Agent E


**Iteration 9**, Agent B updates according to "TestMessage", since there are two "TestMessage" sent to B,
The function "receive_test_message" is called twice

<img src="figs/Iteration9.PNG" width="70%" height="70%">

We can continue to demo each iteration, but due to time limits, let us stop here. As you can see, if we model our agent and message carefully, we can accurately model a nuclear fission process and even calculate the amount of energy released, this is the power of an agent based modeling.
as we just went through how to use different functions in scheduler to move the clock forward event by event.
Couple of things to remember:
1. tick may or may not advance the clock, as it depends on the alarmed messages in the schedule
2. At one point of time t = x, multiple iterations/updates can happen 

**Is there a way to let this "chain reaction" happen automatically?**
<br><img src="figs/chain_reaction.gif" width="30%" height="30%">

## Understanding scheduler.run()

While analysing the scheduler clock step by step, we saw that scheduler relies on "tick" to decide what time is it for next iteration. Each iteration is a completion of scheduler kernel

<img src="figs/MasLiteSchedulerKernel.PNG" width="60%" height="60%">

The functions we used to run step by step has been well capsulated in **Scheduler.run()**. Let's see it in action below. We use the exact same example as in previous section.

In [66]:
TestAgent.agents = [] # clear this class attribute
random.seed(7) # set up a random seed note we set the random seed to 7 instead of 1!
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0, 0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [67]:
scheduler.run()
assert scheduler.clock.time == 4

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent A at t = 0, alarm time is 3, receiver is Agent B
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent A
Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 0 alarms
Agent B receives alarm From A to B, created at 0, scheduled alarm time 3, decides to set 2 alarms
An alarm has been set by Agent B at t = 3, alarm time is 0, receiver is Agent E
An alarm has been set by Agent B at t = 3, alarm time is 1, receiver is Agent E
Agent E receives alarm From B to E, created at 3, scheduled alarm time 0, decides to set 0 alarms
Agent E receives alarm From B to E, created at 3, scheduled alarm time 1, decides to set 0 alarms


**Could you try to recover what happened at each iteration and how many iterations based on the printout?**

Without any parameter tuning, the scheduler.run() will run until "heat death of the universe", meaning there is no alarms/messages scheduled anymore. The simulation comes to an end. However, we can fine tune our scheduler to run to well controlled number of iterations or to a certain time, and continue later. Thus, inside **schedule.run()**, we are going to focus on two important parameters **iterations** and **seconds**. Let's look at **iterations** parameter first.

### Parameter : iterations

Now, each **iteration** in the scheduler corresponds to n Maslite Kernel operation.
Let's understand this by analyzing **3 iterations** and **5 iterations** of scheduler respectively.

In [68]:
TestAgent.agents = [] # clear this class attribute
random.seed(1) # set up a random seed, note we set the random seed to 7 instead of 1!
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0, 0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [69]:
scheduler.run(iterations=3)

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 1 alarms
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent E
Agent E receives alarm From A to E, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent A
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent D
Agent D receives alarm From E to D, created at 0, scheduled alarm time 3, decides to set 3 alarms
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent B
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent D
An alarm has been set by Agent D at t = 3, alarm time is 3, receiver is Agent D
Agent A receives alarm From E to A, created at 0, scheduled alarm time 3, decides to set 0 alarms


**iterations 1:** t = 0, Agent A receives message that were sent to itself and randomly selects Agent E and sets alarm for it with alarm time = 0.

**iterations 2:** t = 0, Agent E receives the alarm and randomly selects Agent A and D and sets alarm for them with alarm time = 3

**iterations 3:** t = 3, Agent A receives the alarm and randomly selects Agent B and D and sets alarms. Agent D receives the alarm and decides not to set any alarms "number of alarms to set:  0"

We can let the scheduler continue to run!

In [70]:
scheduler.run(iterations=5)

{'B', 'D'}
Agent B receives alarm From D to B, created at 3, scheduled alarm time 0, decides to set 3 alarms
An alarm has been set by Agent B at t = 3, alarm time is 5, receiver is Agent C
An alarm has been set by Agent B at t = 3, alarm time is 4, receiver is Agent B
An alarm has been set by Agent B at t = 3, alarm time is 2, receiver is Agent A
Agent D receives alarm From D to D, created at 3, scheduled alarm time 0, decides to set 0 alarms
Agent A receives alarm From B to A, created at 3, scheduled alarm time 2, decides to set 0 alarms
Agent B receives alarm From B to B, created at 3, scheduled alarm time 4, decides to set 0 alarms
Agent C receives alarm From B to C, created at 3, scheduled alarm time 5, decides to set 0 alarms


**iterations 4:** t = 3, Agent D receives alarm set by A and decides to set 3 alarms for C, B, A with different alarm time.
Agent B receives alarm set by A, but decides not to set any alarm
    
**iterations 5:** Agent A receives alarm set by D but decides not to set any alarm

**iterations 6:** Agent B receives alarm set by D but decides not to set any alarm

**iterations 7:** Agent C receives alarm set by D but decides not to set any alarm

**iterations 8: Where is iteration 8?**


### parameter: seconds

Let's call the time when alarm(s) is(are) scheduled to go off "point of interest". Then at each iteration, scheduler tries to advance the clock to the nearest "point of interest" in the future. If we put a "seconds" constraint inside scheduler.run() function, then scheduler will not advance the clock if the current time has passed the scheduled run time (time before the run starts + "seconds")

In [72]:
TestAgent.agents = [] # clear this class attribute
random.seed(1) # set up a random seed, note we set the random seed to 7 instead of 1!
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0, 0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [73]:
scheduler.run(seconds=3)

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 1 alarms
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent E
Agent E receives alarm From A to E, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent A
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent D


we can clearly see that only 2 iterations were executed, and the scheduler's time is 3

In [75]:
assert scheduler.clock.time == 3

In [77]:
TestAgent.agents = [] # clear this class attribute
random.seed(1) # set up a random seed, note we set the random seed to 7 instead of 1!
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0,  0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [78]:
scheduler.run(seconds=5)

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 1 alarms
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent E
Agent E receives alarm From A to E, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent A
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent D
Agent D receives alarm From E to D, created at 0, scheduled alarm time 3, decides to set 3 alarms
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent B
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent D
An alarm has been set by Agent D at t = 3, alarm time is 3, receiver is Agent D
Agent A receives alarm From E to A, created at 0, scheduled alarm time 3, decides to set 0 alarms
Agent B receives alarm From D to B, created at 3, scheduled alarm time 0, decides to set 3 alarms
An alarm has been set by Agent

In [80]:
assert scheduler.clock.time == 5

In [81]:
TestAgent.agents = [] # clear this class attribute
random.seed(1) # set up a random seed, note we set the random seed to 7 instead of 1!
agents = [TestAgent(i) for i in ['A', 'B', 'C', 'D', 'E']] # let us create 5 agents
scheduler = Scheduler(real_time = False) # remember the real_time !
for agent in agents:
    scheduler.add(agent)
agents[0].send(TestMessage('A', 'A', 0,  0))  # let us prime agent A with a test message

Registering agent TestAgent A
A subscribing to A on all topics.
A subscribing to TestAgent for all agents.
Registering agent TestAgent B
B subscribing to B on all topics.
B subscribing to TestAgent for all agents.
Registering agent TestAgent C
C subscribing to C on all topics.
C subscribing to TestAgent for all agents.
Registering agent TestAgent D
D subscribing to D on all topics.
D subscribing to TestAgent for all agents.
Registering agent TestAgent E
E subscribing to E on all topics.
E subscribing to TestAgent for all agents.


In [82]:
scheduler.run(seconds=10)

Agent A receives alarm From A to A, created at 0, scheduled alarm time 0, decides to set 1 alarms
An alarm has been set by Agent A at t = 0, alarm time is 0, receiver is Agent E
Agent E receives alarm From A to E, created at 0, scheduled alarm time 0, decides to set 2 alarms
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent A
An alarm has been set by Agent E at t = 0, alarm time is 3, receiver is Agent D
Agent D receives alarm From E to D, created at 0, scheduled alarm time 3, decides to set 3 alarms
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent B
An alarm has been set by Agent D at t = 3, alarm time is 0, receiver is Agent D
An alarm has been set by Agent D at t = 3, alarm time is 3, receiver is Agent D
Agent A receives alarm From E to A, created at 0, scheduled alarm time 3, decides to set 0 alarms
Agent B receives alarm From D to B, created at 3, scheduled alarm time 0, decides to set 3 alarms
An alarm has been set by Agent

In [84]:
scheduler.clock.time

11